In [18]:
import os
import shutil
import random
import tensorflow as tf
import matplotlib.pyplot as plt

In [25]:
sess = tf.Session()
if sess:
    devices = sess.list_devices()
    for d in devices:
        print(d.name)

    session.close()
    session.__del__()

/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0


In [44]:
work_dir = 'data'
image_height, image_width = 150, 150
train_dir = os.path.join(work_dir, 'data', 'train')
test_dir = os.path.join(work_dir, 'data', 'test')
no_classes = 2
no_validation = 80
epochs = 2
batch_size = 2
no_train = 200
no_test = 80
input_shape = (image_height, image_width, 3)
epoch_steps = no_train // batch_size
test_steps = no_test // batch_size
image_names = sorted(os.listdir(os.path.join(work_dir, 'train')))

In [46]:
def copy_files(prefix_str, range_start, range_end, target_dir):
    image_paths = [os.path.join(work_dir, 'train', prefix_str + '.' + str(i) + '.jpg') for i in range(range_start, range_end)]
    dest_dir = os.path.join(work_dir, 'data', target_dir, prefix_str)
    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)
    os.makedirs(dest_dir)
    for image_path in image_paths:
        shutil.copy(image_path, dest_dir)
        
copy_files('dog', 0, 100, 'train')
copy_files('cat', 0, 100, 'train')
copy_files('dog', 100, 140, 'test')
copy_files('cat', 100, 140, 'test')

In [47]:
generator_train = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
generator_test = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_images = generator_train.flow_from_directory(train_dir,
                                                   batch_size=batch_size,
                                                   target_size=(image_width, image_height))
test_images = generator_test.flow_from_directory(test_dir,
                                                 batch_size=batch_size,
                                                 target_size=(image_width, image_height))

Found 200 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [48]:
def simple_cnn(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(filters=64,
                                    kernel_size=(3, 3),
                                    activation='relu',
                                    input_shape=input_shape))
    model.add(tf.keras.layers.Conv2D(filters=128,
                                    kernel_size=(3, 3),
                                    activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=1024, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Dense(units=no_classes, activation='softmax'))
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

simple_cnn_model = simple_cnn(input_shape)

In [49]:
simple_cnn_model.fit_generator(train_images,
                               steps_per_epoch=epoch_steps,
                               epochs=epochs,
                               validation_data=test_images,
                               validation_steps=test_steps)

ResourceExhaustedError: OOM when allocating tensor of shape [682112,1024] and type float
	 [[node training_2/Adam/zeros_4 (defined at <ipython-input-49-c45afd6a57c7>:5) ]]

Caused by op 'training_2/Adam/zeros_4', defined at:
  File "C:\Users\VALERIO\.conda\envs\dl\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\VALERIO\.conda\envs\dl\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\VALERIO\.conda\envs\dl\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\VALERIO\.conda\envs\dl\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\VALERIO\.conda\envs\dl\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "C:\Users\VALERIO\.conda\envs\dl\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "C:\Users